In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import operator
import seaborn as sns
import json
import matplotlib.pyplot as plt

In [4]:
def load_json_to_df(datapass):
    data = [] 
    with open(datapass) as data_file: 
        for f in data_file:
            data.append(json.loads(f))
    df = pd.DataFrame(data)
    return df

In [5]:
business = load_json_to_df("business.json")

In [6]:
review = load_json_to_df("review.json")

In [ ]:
user = load_json_to_df("user.json")

In [7]:
n_users_review = review.user_id.unique().shape[0]
n_items_review = review.business_id.unique().shape[0]

In [33]:
#Group res by city
city = business.groupby('city')['city'].count()
city
# subset restaurant to category restaurant
#restaurant = business[business['categories'].str.contains("Restaurants",na=False)]
is_rest = []
for i in business['categories']:
    
    if 'Restaurants' in i or 'Food' in i:
        is_rest.append(True)
    else:
        is_rest.append(False)
restaurant = business.loc[is_rest]

In [34]:
city2 = restaurant.groupby('city')['city'].count()
city.head(2)

city
                 3
110 Las Vegas    1
Name: city, dtype: int64

In [36]:
restaurant.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
1,2824 Milton Rd,"{'GoodForMeal': {'dessert': False, 'latenight'...",mLwM-h2YhXl2NCgdS84_Bw,"[Food, Soul Food, Convenience Stores, Restaura...",Charlotte,"{'Monday': '10:00-22:00', 'Tuesday': '10:00-22...",0,35.236870,-80.741976,South Florida Style Chicken & Ribs,Eastland,28215,4,4.5,NC
2,337 Danforth Avenue,"{'BusinessParking': {'garage': False, 'street'...",v2WhjAB3PIBA8J8VxG3wEg,"[Food, Coffee & Tea]",Toronto,"{'Monday': '10:00-19:00', 'Tuesday': '10:00-19...",0,43.677126,-79.353285,The Tea Emporium,Riverdale,M4K 1N7,7,4.5,ON


In [38]:
res_vegas = restaurant.loc[restaurant['city']=='Las Vegas']
res_vegas.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
58,2400 E Lake Mead Blvd,"{'RestaurantsTableService': True, 'GoodForMeal...",LDMCrFlGIFUN6L-FEFgzWg,"[Restaurants, American (Traditional), Mexican,...",Las Vegas,"{'Monday': '9:00-23:00', 'Tuesday': '9:00-23:0...",1,36.196203,-115.116799,El Pollo Loco,,89030,12,3.0,NV
64,4190 S Rainbow Blvd,"{'RestaurantsPriceRange2': 1, 'RestaurantsAtti...",QTH_XGh4rWYdd0fTW-tUDw,"[Mexican, Restaurants]",Las Vegas,{},0,36.112519,-115.242605,Baja Fresh Mexican Grill,Spring Valley,89103,7,3.5,NV


In [39]:
# Left join restaruants in toronto table with review table
review_rest_tor = pd.merge(res_vegas, review, on='business_id', how='left')
# review_rest_tor.shape
# review_rest_tor.columns

# Subset to user, item, rating columns
uir = review_rest_tor[['user_id','business_id','stars_y']]
# len(uir)

# Assign index for user and item
user_index = uir.user_id.unique()
item_index = uir.business_id.unique()

# Count number of unique users and items
n_users = uir.user_id.unique().shape[0]
n_items = uir.business_id.unique().shape[0]

In [ ]:
# Split User, Item, Rating dataset to train and test sets of 70% & 30%
from sklearn.model_selection import train_test_split
train, test = train_test_split(uir, test_size=0.30, random_state=42)
# len(train)
# len(test)

# Create table for train data with list of users as index & items as columns
train_matrix = pd.DataFrame(index=user_index, columns=item_index)
# train_matrix.shape

# Fill in train_matrix table with ratings
for row in train.itertuples():
    user = row[1]
    item = row[2]
    train_matrix.loc[user][item] = row[3]  

# Create table for test data with list of users as index & items as columns    
test_matrix = pd.DataFrame(index=user_index, columns=item_index)
# test_matrix.shape

# Fill in test_matrix table with ratings
for row in test.itertuples():
    user = row[1]
    item = row[2]
    test_matrix.loc[user][item] = row[3]

# Begin filtering process to create 5 Core Subset

# Count number of rated items for each user
item_1 = train_matrix.apply(lambda x: x > 0, raw=True).sum(axis=1)
# item_1.value_counts()

# Filter down to the users with greater than or equal to 5 ratings
train1 = train_matrix
train1['item_1'] = item_1
train2 = train1.loc[train1['item_1'] >= 5]
# train2.shape

# Count number of rated users for each item
train2 = train2.drop('item_1',axis=1)
train3 = train2.transpose()
user_1 = train3.apply(lambda x: x > 0, raw=True).sum(axis=1)
# user_1.value_counts()

# Filter down to the items with greater than or equal to 5 ratings
train3['user_1'] = user_1
train4 = train3.loc[train3['user_1'] >= 5]
train4 = train4.drop('user_1',axis=1)
train5 = train4.transpose()
# train5.shape

# Repeat the process for both user and item
item_2 = train5.apply(lambda x: x > 0, raw=True).sum(axis=1)
# item_2.value_counts()
# item_2.shape
train5['item_2'] = item_2
train6 = train5.loc[train5['item_2'] >= 5]
train6 = train6.drop('item_2',axis=1)
train7 = train6.transpose()
user_2 = train7.apply(lambda x: x > 0, raw=True).sum(axis=1)
# user_2.value_counts()
train7['user_2'] = user_2
train8 = train7.loc[train7['user_2'] >= 5]
train8 = train8.drop('user_2',axis=1)
train9 = train8.transpose()
# train9.shape

# Check every user and item has at least 5 ratings
item_3 = train9.apply(lambda x: x > 0, raw=True).sum(axis=1)
# item_3.value_counts()
user_3 = train9.apply(lambda x: x > 0, raw=True).sum(axis=1)
# user_3.value_counts()

# Filter down the test matrix to filtered user and item in train matrix
test9 = test_matrix.loc[train9.index,train9.columns]
# test9.shape

In [ ]:
print '\nYelp Review Dataset'
print 'Size of original Yelp dataset, Review is %s ' % str(review.shape)
print 'Number of users is %s, number of items is %s ' % (str(n_users_review),str(n_items_review))
print '\nYelp Business Dataset'
print 'Size of original Yelp dataset, Business is %s ' % str(business.shape)
print 'Size of Restaurants subset of Business dataset is %s ' % str(restaurant.shape)
print '\nTop 10 Cities with the most number of restaurants are as follows \n%s ' % str(city2.sort_values(ascending=False).head(10))
print '\nRestaurants in Toronto Subset'
print 'Size of Restaurants in Toronto subset of Business dataset is %s ' % str(restaurant_toronto.shape)
print 'User, Item, Rating dataset for restuarnats in Toronto contain %s rows ' % str(len(uir))
print 'Number of users is %s, number of items is %s ' % (str(n_users),str(n_items))
print '\nRestaurants in Phoenix Subset'
print 'Size of Restaurants in Phoenix subset of Business dataset is %s ' % str(restaurant_phoenix.shape)
print 'User, Item, Rating dataset for restuarnats in Toronto contain %s rows ' % str(len(uir_ph))
print 'Number of users is %s, number of items is %s ' % (str(n_users_ph),str(n_items_ph))

In [ ]:
print '\nRestaurants in Toronto'
print '\nTrain dataset contains %s rows, Test dataset contains %s rows ' % (str(len(train)),str(len(test)))
print 'Size of train & test matrices with users in rows & items in columns is %s ' % (str(train_matrix.shape))
print '\nUsers of filtered 5 core subset has rated at least 5 items as follows  \n%s' % (str(item_3.value_counts().head(5)))
print '\nItems of filtered 5 core subset has been rated by at least 5 users as follows  \n%s' % (str(user_3.value_counts().head(5)))
print '\nSize of 5 core subset is %s ' % (str(train9.shape))
print '\nRestaurants in Phoenix'
print '\nTrain dataset contains %s rows, Test dataset contains %s rows ' % (str(len(train_ph)),str(len(test_ph)))
print 'Size of train & test matrices with users in rows & items in columns is %s ' % (str(train_matrix_ph.shape))
print '\nSize of 5 core subset is %s ' % (str(train9_ph.shape))

In [41]:
def collaborative_filtering(train, test, sim='cosine', type='user', knn=5):

    # Fill NAN values in train & test data as 0's
    train_0 = train.fillna(0)
    test_0 = test.fillna(0)

    # Create a similarity matrix of either users or items based on cosine or 
    # pearson correlation measure
    if sim == 'cosine':
        user_dist = pairwise_distances(train_0, metric='cosine')
        item_dist = pairwise_distances(train_0.T, metric='cosine')

    elif sim == 'pearson':
        user_dist = pairwise_distances(train_0, metric='correlation')
        item_dist = pairwise_distances(train_0.T, metric='correlation')
        
    user_sim = 1 - user_dist
    item_sim = 1 - item_dist
 
    # Create a dataframe with mean user ratings
    mean_rating = train.mean(axis=1)
    mean_user_rating = pd.concat([mean_rating] * len(train.columns), axis=1)
    mean_user_rating.columns = train.columns
    
    # Modify a dataframe so that mean user ratings are present only in matrix 
    # positions of rated items and 0's in matrix positions of non-rated items
    mean_user_rating_0 = mean_user_rating
    mean_user_rating_0[train_0 == 0] = 0
    
    # Normalize every user's ratings to mean of zero
    ratings_diff = train_0 - mean_user_rating_0
    
    # Create a dataframe with user's mean user ratings present in all items
    mean_user_rating_f = pd.concat([mean_rating] * len(train.columns), axis=1)
    mean_user_rating_f.columns = train.columns
    
    # Create a placeholder dataframe for predictions of rated items in test data
    pred = pd.DataFrame(index=train.index, columns=train.columns)
    
    # User-Based Collaborative Filtering
    if type == 'user':    
        # Index user similarity matrix with user ids for both rows and columns
        user_sim = pd.DataFrame(user_sim, index=train.index, columns=train.index)
        # When the number of k neareast neighbors is specified
        if knn != 'all':
            user_sim_mat = user_sim.as_matrix()
        
            # Item id's of rated items in test data for each user
            cols = test.columns
            test_rated = test.apply(lambda x: x > 0, raw=True).apply(lambda x: list(cols[x.values]), axis=1)
            
            # Iterate over each user, m
            for m in range(len(user_sim)):
                # Retrieve column of user m's similarities to all other users
                temp = user_sim_mat[m]
                temp = pd.DataFrame(temp,index=train.index, columns=['similarity'])
                # Rank user m's similarities
                temp['rank'] = temp['similarity'].rank(ascending=0)
                
                # Iterate over user m's rated items
                for n in range(len(test_rated[m])):
                    # For user m's nth rated item, extract column of ratings of 
                    # all users corresponding to nth item
                    temp2 = ratings_diff[[test_rated[m][n]]]
                    temp2.columns = ['rating']
                    # Contatenate similarity, rank, rating as one dataframe
                    result = pd.concat([temp, temp2], axis=1)    
                    # Filter down to the users who rated the items
                    result2 = result[result['rating'] != 0]
                    # Filter down to knn number of users with the knn 
                    # highest similairites
                    result3 = result2.nsmallest(int(knn), 'rank')
                
                    # Divide weighted sum of user's knn nearest neighbors' ratings by 
                    # sum of their similarities
                    score = result3['similarity'].dot(result3['rating'])/result3['similarity'].sum()
                    mean = mean_user_rating_f.loc[train.index[m]][test_rated[m][n]]
                
                    # Make a prediction by adding user's mean rating to weighted sum
                    pred.loc[train.index[m]][test_rated[m][n]] = mean + score
                    
        # When the number of k neareast neighbors is not specified and all 
        # available neighbors are used for prediction     
        elif knn == 'all':  
            # Compute user similarity weighted sum of available ratings of 
            # user's every neighbor 
            num_user = user_sim.dot(ratings_diff)
            
            # Sum user similarities    
            sum_sim_user = user_sim.sum(axis=1)
            sum_sim_mat_user = pd.concat([sum_sim_user] * len(train.columns), axis=1)
            sum_sim_mat_user.columns = train.columns

            # Create a dataframe of predictions computed by adding mean user 
            # rating to user similairty weighted sum of user's ratings 
            # divided by sum of user similarities
            pred = mean_user_rating_f + num_user / sum_sim_mat_user
                
    # Item-Based Collaborative Filtering  
    elif type == 'item':
        # Index item similarity matrix with item ids for both rows and columns
        item_sim = pd.DataFrame(item_sim, index=train.columns, columns=train.columns)
        
        # When the number of k neareast neighbors is specified
        if knn != 'all':
            item_sim_mat = item_sim.as_matrix()

            # User id's of rated items in test data for each item
            cols2 = test.T.columns
            test_rated2 = test.T.apply(lambda x: x > 0, raw=True).apply(lambda x: list(cols2[x.values]), axis=1)
        
            # Iterate over each item, m
            for m in range(len(item_sim)):
                # Retrieve column of item m's similarities to all other items
                temp = item_sim_mat[m]
                temp = pd.DataFrame(temp,index=train.columns, columns=['similarity'])
                # Rank item m's similarities
                temp['rank'] = temp['similarity'].rank(ascending=0)

                # Iterate over item m's rated users
                for n in range(len(test_rated2[m])):
                    # For item m's nth rated user, extract column of ratings of 
                    # all items corresponding to nth user
                    temp2 = ratings_diff.T[[test_rated2[m][n]]]
                    temp2.columns = ['rating']
                    # Contatenate similarity, rank, rating as one dataframe
                    result = pd.concat([temp, temp2], axis=1)    
                    # Filter down to the items that are rated by the user
                    result2 = result[result['rating'] != 0]
                    # Filter down to knn number of items with the knn 
                    # highest similairites
                    result3 = result2.nsmallest(int(knn), 'rank')
                
                    # Divide weighted sum of item's knn nearest neighbors' 
                    # ratings by sum of their similarities
                    score = result3['similarity'].dot(result3['rating'])/result3['similarity'].sum()
                    mean = mean_user_rating_f.loc[test_rated2[m][n]][train.columns[m]]
                
                    # Make a prediction by adding user's mean rating to weighted sum
                    pred.loc[test_rated2[m][n]][train.columns[m]] = mean + score
        
        # When the number of k neareast neighbors is not specified and 
        # all available neighbors are used for prediction
        elif knn == 'all':
            # Compute item similarity weighted sum of available ratings of 
            # item's every neighbor
            num_item = ratings_diff.dot(item_sim)

            # Sum item similarities 
            sum_sim_item = item_sim.sum(axis=1)
            sum_sim_item = pd.DataFrame(sum_sim_item, index=train.columns)
            sum_sim_mat_item = pd.concat([sum_sim_item.T] * len(train), axis=0)

            sum_sim_mat_item.index = train.index

            # Create a dataframe of predictions computed by adding mean user 
            # rating to item similairty weighted sum of items' ratings 
            # divided by sum of item similarities
            pred = mean_user_rating_f + num_item / sum_sim_mat_item
            
    pred_0 = pred.fillna(0)
    pred_0_mat = pred_0.as_matrix()            
    return pred_0_mat

In [ ]:
test9_0 = test9.fillna(0)
test9_0_mat = test9_0.as_matrix()


In [ ]:
from sklearn.metrics import mean_absolute_error
def mae(prediction, actual):
    prediction = prediction[actual.nonzero()].flatten() 
    actual = actual[actual.nonzero()].flatten()
    return mean_absolute_error(prediction, actual)

In [ ]:

start_time = time.time()
pred_user_5 = collaborative_filtering(train9, test9, sim='cosine', type='user', knn='5')